<a href="https://colab.research.google.com/github/JefersonCardona/Algoritmia-Programacion/blob/main/JefCar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import os
from datetime import datetime
import logging

In [ ]:
RutaFinal = 'Ruta_final'
if not os.path.exists(RutaFinal):
    os.makedirs(RutaFinal)

ArchivoLog = os.path.join(RutaFinal, 'asignaciones.log')

MensajesLog = []

def MensajeLog(L, Mensaje):
    MensajesLog.append(f"{datetime.now()} - {L} - {Mensaje}")
    if L == "INFO":
        print(f"{datetime.now()} - INFO - {Mensaje}")
    elif L == "ERROR":
        print(f"{datetime.now()} - ERROR - {Mensaje}")

MensajeLog("INFO", "Cargando datos de Estudiantes y Malla Curricular")
Datos = pd.read_csv('https://github.com/JefersonCardona/Algoritmia-Programacion/raw/eb776f421bb990b8c16ded477572f5d436f1f646/Archivos/Estudiantes.csv', encoding='latin1', delimiter=';').drop(columns=['Fecha'])
MallaCurricular = pd.read_csv('https://github.com/JefersonCardona/Algoritmia-Programacion/raw/eb776f421bb990b8c16ded477572f5d436f1f646/Archivos/MallaCurricular.csv', encoding='latin1', delimiter=';')
MensajeLog("INFO", "Datos cargados con éxito")

def AsignaturaCode(asignatura, semestre, creditos, consecutivo):
    return f"{asignatura[:3].upper()}{semestre}{creditos}{consecutivo:02d}"

def HTD(creditos):
    if creditos == 4:
        return 96
    elif creditos == 3:
        return 64
    elif creditos == 2:
        return 32
    elif creditos == 1:
        return 16
    return 0

def HTI(creditos):
    if creditos == 4:
        return 120
    elif creditos == 3:
        return 80
    elif creditos == 2:
        return 64
    elif creditos == 1:
        return 32
    return 0

def FuncionPrincipal(estudiantes_df, asignaturas_df, semestre, nivel, tamano_grupo):
    MensajeLog("INFO", f"Matriculando estudiantes para el semestre {semestre}, nivel {nivel}")
    estudiantes_semestre = estudiantes_df[estudiantes_df['Semestre'] == semestre]
    total_estudiantes_semestre = len(estudiantes_semestre)
    asignaturas_nivel = asignaturas_df[asignaturas_df['Nivel'] == nivel]
    grupos = [estudiantes_semestre[i:i + tamano_grupo] for i in range(0, len(estudiantes_semestre), tamano_grupo)]

    Directorio = os.path.join(RutaFinal, f'Asignaciones_Semestre_{semestre}')
    if not os.path.exists(Directorio):
        os.makedirs(Directorio)
        MensajeLog("INFO", f"Directorio creado: {Directorio}")

    for index, asignatura in asignaturas_nivel.iterrows():
        asignatura_nombre = asignatura['Asignatura']
        creditos = asignatura['Creditos']
        total_cursos_asignados = len(grupos)
        fecha_creacion = datetime.now().strftime('%Y%m%d')
        asignatura_dir = os.path.join(Directorio, asignatura_nombre)
        if not os.path.exists(asignatura_dir):
            os.makedirs(asignatura_dir)
            MensajeLog("INFO", f"Directorio creado: {asignatura_dir}")

        for numero_grupo, grupo in enumerate(grupos, start=1):
            codigo_asignatura = AsignaturaCode(asignatura_nombre, semestre, creditos, numero_grupo)
            horas_trabajo_docente = HTD(creditos)
            horas_trabajo_independiente = HTI(creditos)
            numero_total_estudiantes = total_estudiantes_semestre
            cantidad_estudiantes = len(grupo)
            nombre_curso_formateado = asignatura_nombre.replace(" ", "").capitalize()
            grupo_filename = f"{codigo_asignatura}-{nombre_curso_formateado}-{cantidad_estudiantes}-{numero_grupo}.csv"
            grupo_excel_filename = f"{codigo_asignatura}-{nombre_curso_formateado}-{cantidad_estudiantes}-{numero_grupo}.xlsx"
            grupo_path = os.path.join(asignatura_dir, grupo_filename)
            grupo_excel_path = os.path.join(asignatura_dir, grupo_excel_filename)
            grupo_df = pd.DataFrame({'Estudiante': grupo['Nombre'], '(CA)': codigo_asignatura, '(HTD)': horas_trabajo_docente, '(HTI)': horas_trabajo_independiente, '(NTE)': numero_total_estudiantes, '(CC)': numero_grupo, '(TCA)': total_cursos_asignados, '(FC)': fecha_creacion })
            grupo_df.to_csv(grupo_path, index=False, encoding='latin1')
            grupo_df.to_excel(grupo_excel_path, index=False)
            MensajeLog("INFO", f"Archivos generados para la asignatura {asignatura_nombre}, grupo {numero_grupo}")

    MensajeLog("INFO", f"Archivos CSV de asignaciones generados para el semestre {semestre}")


In [ ]:
FuncionPrincipal(Datos, MallaCurricular, semestre=1, nivel=1, tamano_grupo=30)
FuncionPrincipal(Datos, MallaCurricular, semestre=2, nivel=2, tamano_grupo=30)
FuncionPrincipal(Datos, MallaCurricular, semestre=3, nivel=3, tamano_grupo=30)
FuncionPrincipal(Datos, MallaCurricular, semestre=4, nivel=4, tamano_grupo=25)
FuncionPrincipal(Datos, MallaCurricular, semestre=5, nivel=5, tamano_grupo=25)
FuncionPrincipal(Datos, MallaCurricular, semestre=6, nivel=6, tamano_grupo=25)
FuncionPrincipal(Datos, MallaCurricular, semestre=7, nivel=7, tamano_grupo=20)
FuncionPrincipal(Datos, MallaCurricular, semestre=8, nivel=8, tamano_grupo=20)
FuncionPrincipal(Datos, MallaCurricular, semestre=9, nivel=9, tamano_grupo=20)
FuncionPrincipal(Datos, MallaCurricular, semestre=10, nivel=10, tamano_grupo=10)

with open(ArchivoLog, 'w') as log_file:
    for Mensaje in MensajesLog:
        log_file.write(Mensaje + '\n')